In [1]:
from datetime import datetime
from pathlib import Path

import numpy as np
import polars as pl
import pytz
import torch
from datasets import Dataset
from omegaconf import OmegaConf
from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
)
from sentence_transformers.losses import MultipleNegativesRankingLoss
from sentence_transformers.training_args import BatchSamplers
from sklearn.metrics.pairwise import cosine_similarity

from src.config import cfg
from src.data import add_subject_name_info, preprocess_train
from src.dir import create_dir
from src.seed import seed_everything

cfg.exp_number = Path().resolve().name
print(OmegaConf.to_yaml(cfg, resolve=True))

seed_everything(cfg.seed)


/home/marumarukun/pj/compe/kaggle_eedi/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


exp_number: '002'
run_time: base
data:
  input_root: ../../data/input
  train_path: ../../data/input/train.csv
  test_path: ../../data/input/test.csv
  sample_submission_path: ../../data/input/sample_submission.csv
  mapping_path: ../../data/input/misconception_mapping.csv
  mapping_meta_path: ../../data/input/mapping_meta.parquet
  output_root: ../../data/output
  results_root: ../../results
  results_path: ../../results/002/base
seed: 42
k: 25
model:
  model_name: BAAI/bge-large-en-v1.5
  epoch: 2
  lr: 2.0e-05
  batch_size: 8



In [2]:
DEBUG = False


### Data Load

In [3]:
# データの読み込み
train = pl.read_csv(cfg.data.train_path, try_parse_dates=True)
test = pl.read_csv(cfg.data.test_path, try_parse_dates=True)
sample_submission = pl.read_csv(cfg.data.sample_submission_path, try_parse_dates=True)
mapping = pl.read_csv(cfg.data.mapping_path, try_parse_dates=True)
mapping_meta = pl.read_parquet(cfg.data.mapping_meta_path)


In [4]:
# trainの前処理
train_long = preprocess_train(train)
train_long.head()


shape: (5, 11)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ QuestionI ┆ Construct ┆ SubjectNa ┆ QuestionT ┆ … ┆ AllText   ┆ AnswerAlp ┆ QuestionI ┆ Misconce │
│ d         ┆ Name      ┆ me        ┆ ext       ┆   ┆ ---       ┆ habet     ┆ d_Answer  ┆ ptionId  │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ str       ┆ ---       ┆ ---       ┆ ---      │
│ i64       ┆ str       ┆ str       ┆ str       ┆   ┆           ┆ str       ┆ str       ┆ i64      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 0         ┆ Use the   ┆ BIDMAS    ┆ \[        ┆ … ┆ Construct ┆ D         ┆ 0_D       ┆ 1672     │
│           ┆ order of  ┆           ┆ 3 \times  ┆   ┆ Name: Use ┆           ┆           ┆          │
│           ┆ operation ┆           ┆ 2+4-5     ┆   ┆ the order ┆           ┆           ┆          │
│           ┆ s to…     ┆           ┆ \]        ┆   ┆ o…        ┆           ┆           ┆          │
│           ┆           ┆           ┆ Where do  ┆   ┆           ┆           ┆           ┆          │
│           ┆           ┆           ┆ …         ┆   ┆           ┆           ┆           ┆          │
│ 1000      ┆ Simplify  ┆ Simplifyi ┆ Simplify  ┆ … ┆ Construct ┆ A         ┆ 1000_A    ┆ 891      │
│           ┆ an        ┆ ng        ┆ the follo ┆   ┆ Name:     ┆           ┆           ┆          │
│           ┆ algebraic ┆ Algebraic ┆ wing, if  ┆   ┆ Simplify  ┆           ┆           ┆          │
│           ┆ fraction… ┆ Fraction… ┆ pos…      ┆   ┆ an alg…   ┆           ┆           ┆          │
│ 1000      ┆ Simplify  ┆ Simplifyi ┆ Simplify  ┆ … ┆ Construct ┆ C         ┆ 1000_C    ┆ 891      │
│           ┆ an        ┆ ng        ┆ the follo ┆   ┆ Name:     ┆           ┆           ┆          │
│           ┆ algebraic ┆ Algebraic ┆ wing, if  ┆   ┆ Simplify  ┆           ┆           ┆          │
│           ┆ fraction… ┆ Fraction… ┆ pos…      ┆   ┆ an alg…   ┆           ┆           ┆          │
│ 1000      ┆ Simplify  ┆ Simplifyi ┆ Simplify  ┆ … ┆ Construct ┆ D         ┆ 1000_D    ┆ 353      │
│           ┆ an        ┆ ng        ┆ the follo ┆   ┆ Name:     ┆           ┆           ┆          │
│           ┆ algebraic ┆ Algebraic ┆ wing, if  ┆   ┆ Simplify  ┆           ┆           ┆          │
│           ┆ fraction… ┆ Fraction… ┆ pos…      ┆   ┆ an alg…   ┆           ┆           ┆          │
│ 1001      ┆ Round     ┆ Rounding  ┆ What is   ┆ … ┆ Construct ┆ A         ┆ 1001_A    ┆ 1379     │
│           ┆ numbers   ┆ to        ┆ \(        ┆   ┆ Name:     ┆           ┆           ┆          │
│           ┆ to two    ┆ Decimal   ┆ \mathbf{3 ┆   ┆ Round     ┆           ┆           ┆          │
│           ┆ decimal   ┆ Places    ┆ . 5 1 6   ┆   ┆ numbers   ┆           ┆           ┆          │
│           ┆ p…        ┆           ┆ 3…        ┆   ┆ t…        ┆           ┆           ┆          │
└───────────┴───────────┴───────────┴───────────┴───┴───────────┴───────────┴───────────┴──────────┘

In [5]:
# trainのSubjectName情報をmappingに追加
mapping_meta = add_subject_name_info(train, mapping)
mapping_meta.head()


shape: (5, 4)
┌─────────────────┬──────────────────────────┬──────────────────────────┬──────────────────────────┐
│ MisconceptionId ┆ MisconceptionName        ┆ SubjectNames             ┆ MisconceptionName_with_S │
│ ---             ┆ ---                      ┆ ---                      ┆ ubject…                  │
│ i64             ┆ str                      ┆ list[str]                ┆ ---                      │
│                 ┆                          ┆                          ┆ str                      │
╞═════════════════╪══════════════════════════╪══════════════════════════╪══════════════════════════╡
│ 0               ┆ Does not know that       ┆ ["Angles in Triangles"]  ┆ The misconception 'Does  │
│                 ┆ angles in a…             ┆                          ┆ not kn…                  │
│ 1               ┆ Uses dividing fractions  ┆ ["Multiplying and        ┆ The misconception 'Uses  │
│                 ┆ method…                  ┆ Dividing Neg…            ┆ dividi…                  │
│ 2               ┆ Believes there are 100   ┆ ["Types, Naming and      ┆ The misconception        │
│                 ┆ degrees…                 ┆ Estimating…              ┆ 'Believes th…            │
│ 3               ┆ Thinks a quadratic       ┆ ["Factorising into a     ┆ The misconception        │
│                 ┆ without a n…             ┆ Single Br…               ┆ 'Thinks a qu…            │
│ 4               ┆ Believes addition of     ┆ ["Simplifying            ┆ The misconception        │
│                 ┆ terms and…               ┆ Expressions by C…        ┆ 'Believes ad…            │
└─────────────────┴──────────────────────────┴──────────────────────────┴──────────────────────────┘

### Make retrieval data

In [6]:
model = SentenceTransformer(cfg.model.model_name, trust_remote_code=True)

train_long_vec = model.encode(train_long["AllText"].to_list(), normalize_embeddings=True)
misconception_mapping_vec = model.encode(
    mapping_meta["MisconceptionName_with_SubjectNames"].to_list(), normalize_embeddings=True
)
print(train_long_vec.shape, misconception_mapping_vec.shape)


(4370, 1024) (2587, 1024)


In [7]:
# コサイン類似度を計算
train_cos_sim_arr = cosine_similarity(train_long_vec, misconception_mapping_vec)
print(f"train_cos_sim_arr.shape: {train_cos_sim_arr.shape}")
print(train_cos_sim_arr[0])

# コサイン類似度が降順になるように各行をソート
train_sorted_indices = np.argsort(-train_cos_sim_arr, axis=1)
print(f"\ntrain_sorted_indices.shape: {train_sorted_indices.shape}")
print(train_sorted_indices[0])


train_cos_sim_arr.shape: (4370, 2587)
[0.5054938  0.6122866  0.42998317 ... 0.44713372 0.64220685 0.6939039 ]

train_sorted_indices.shape: (4370, 2587)
[2488  871 1316 ... 2299  237  211]


In [8]:
train_long = train_long.with_columns(
    pl.Series(train_sorted_indices[:, : cfg.k].tolist()).alias("PredictMisconceptionId")
)
train_long.head(1)


shape: (1, 12)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ QuestionI ┆ Construct ┆ SubjectNa ┆ QuestionT ┆ … ┆ AnswerAlp ┆ QuestionI ┆ Misconcep ┆ PredictM │
│ d         ┆ Name      ┆ me        ┆ ext       ┆   ┆ habet     ┆ d_Answer  ┆ tionId    ┆ isconcep │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ tionId   │
│ i64       ┆ str       ┆ str       ┆ str       ┆   ┆ str       ┆ str       ┆ i64       ┆ ---      │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ list[i64 │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ ]        │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 0         ┆ Use the   ┆ BIDMAS    ┆ \[        ┆ … ┆ D         ┆ 0_D       ┆ 1672      ┆ [2488,   │
│           ┆ order of  ┆           ┆ 3 \times  ┆   ┆           ┆           ┆           ┆ 871, …   │
│           ┆ operation ┆           ┆ 2+4-5     ┆   ┆           ┆           ┆           ┆ 2181]    │
│           ┆ s to…     ┆           ┆ \]        ┆   ┆           ┆           ┆           ┆          │
│           ┆           ┆           ┆ Where do  ┆   ┆           ┆           ┆           ┆          │
│           ┆           ┆           ┆ …         ┆   ┆           ┆           ┆           ┆          │
└───────────┴───────────┴───────────┴───────────┴───┴───────────┴───────────┴───────────┴──────────┘

In [9]:
mapping_meta.rename(lambda x: "Predict" + x)


shape: (2_587, 4)
┌────────────────────────┬────────────────────────┬────────────────────────┬───────────────────────┐
│ PredictMisconceptionId ┆ PredictMisconceptionNa ┆ PredictSubjectNames    ┆ PredictMisconceptionN │
│ ---                    ┆ me                     ┆ ---                    ┆ ame_with_…            │
│ i64                    ┆ ---                    ┆ list[str]              ┆ ---                   │
│                        ┆ str                    ┆                        ┆ str                   │
╞════════════════════════╪════════════════════════╪════════════════════════╪═══════════════════════╡
│ 0                      ┆ Does not know that     ┆ ["Angles in            ┆ The misconception     │
│                        ┆ angles in a…           ┆ Triangles"]            ┆ 'Does not kn…         │
│ 1                      ┆ Uses dividing          ┆ ["Multiplying and      ┆ The misconception     │
│                        ┆ fractions method…      ┆ Dividing Neg…          ┆ 'Uses dividi…         │
│ 2                      ┆ Believes there are 100 ┆ ["Types, Naming and    ┆ The misconception     │
│                        ┆ degrees…               ┆ Estimating…            ┆ 'Believes th…         │
│ 3                      ┆ Thinks a quadratic     ┆ ["Factorising into a   ┆ The misconception     │
│                        ┆ without a n…           ┆ Single Br…             ┆ 'Thinks a qu…         │
│ 4                      ┆ Believes addition of   ┆ ["Simplifying          ┆ The misconception     │
│                        ┆ terms and…             ┆ Expressions by C…      ┆ 'Believes ad…         │
│ …                      ┆ …                      ┆ …                      ┆ …                     │
│ 2582                   ┆ When multiplying       ┆ []                     ┆ The misconception is: │
│                        ┆ numbers with …         ┆                        ┆ When mul…             │
│ 2583                   ┆ Does not know what a   ┆ ["Square Roots, Cube   ┆ The misconception     │
│                        ┆ cube numb…             ┆ Roots, et…             ┆ 'Does not kn…         │
│ 2584                   ┆ Believes that any      ┆ []                     ┆ The misconception is: │
│                        ┆ percentage o…          ┆                        ┆ Believes…             │
│ 2585                   ┆ Believes a cubic       ┆ ["Expanding Triple     ┆ The misconception     │
│                        ┆ expression sh…         ┆ Brackets an…           ┆ 'Believes a …         │
│ 2586                   ┆ Misunderstands order   ┆ ["Rearranging Formula  ┆ The misconception     │
│                        ┆ of operat…             ┆ and Equa…              ┆ 'Misundersta…         │
└────────────────────────┴────────────────────────┴────────────────────────┴───────────────────────┘

In [10]:
train_retrieved = (
    train_long.explode("PredictMisconceptionId")
    .join(mapping_meta, on="MisconceptionId")
    .join(mapping_meta.rename(lambda x: "Predict" + x), on="PredictMisconceptionId")
    # MisconceptionIdとPredictMisconceptionIdが同じ行（正解してるもの）を削除
    .filter(pl.col("MisconceptionId") != pl.col("PredictMisconceptionId"))
)
display(train_retrieved.head(3))
print(f"train_retrieved.shape: {train_retrieved.shape}")


shape: (3, 18)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ QuestionI ┆ Construct ┆ SubjectNa ┆ QuestionT ┆ … ┆ Misconcep ┆ PredictMi ┆ PredictSu ┆ PredictM │
│ d         ┆ Name      ┆ me        ┆ ext       ┆   ┆ tionName_ ┆ sconcepti ┆ bjectName ┆ isconcep │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ with_Subj ┆ onName    ┆ s         ┆ tionName │
│ i64       ┆ str       ┆ str       ┆ str       ┆   ┆ ect…      ┆ ---       ┆ ---       ┆ _with_…  │
│           ┆           ┆           ┆           ┆   ┆ ---       ┆ str       ┆ list[str] ┆ ---      │
│           ┆           ┆           ┆           ┆   ┆ str       ┆           ┆           ┆ str      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 0         ┆ Use the   ┆ BIDMAS    ┆ \[        ┆ … ┆ The misco ┆ Answers   ┆ ["Multipl ┆ The misc │
│           ┆ order of  ┆           ┆ 3 \times  ┆   ┆ nception  ┆ order of  ┆ ying and  ┆ onceptio │
│           ┆ operation ┆           ┆ 2+4-5     ┆   ┆ 'Confuses ┆ operation ┆ Dividing  ┆ n        │
│           ┆ s to…     ┆           ┆ \]        ┆   ┆ th…       ┆ s qu…     ┆ Alg…      ┆ 'Answers │
│           ┆           ┆           ┆ Where do  ┆   ┆           ┆           ┆           ┆ ord…     │
│           ┆           ┆           ┆ …         ┆   ┆           ┆           ┆           ┆          │
│ 0         ┆ Use the   ┆ BIDMAS    ┆ \[        ┆ … ┆ The misco ┆ Does not  ┆ ["Linear  ┆ The misc │
│           ┆ order of  ┆           ┆ 3 \times  ┆   ┆ nception  ┆ include   ┆ Equations ┆ onceptio │
│           ┆ operation ┆           ┆ 2+4-5     ┆   ┆ 'Confuses ┆ brackets  ┆ "]        ┆ n 'Does  │
│           ┆ s to…     ┆           ┆ \]        ┆   ┆ th…       ┆ when…     ┆           ┆ not in…  │
│           ┆           ┆           ┆ Where do  ┆   ┆           ┆           ┆           ┆          │
│           ┆           ┆           ┆ …         ┆   ┆           ┆           ┆           ┆          │
│ 0         ┆ Use the   ┆ BIDMAS    ┆ \[        ┆ … ┆ The misco ┆ Does not  ┆ ["BIDMAS" ┆ The misc │
│           ┆ order of  ┆           ┆ 3 \times  ┆   ┆ nception  ┆ understan ┆ ]         ┆ onceptio │
│           ┆ operation ┆           ┆ 2+4-5     ┆   ┆ 'Confuses ┆ d that    ┆           ┆ n 'Does  │
│           ┆ s to…     ┆           ┆ \]        ┆   ┆ th…       ┆ the n…    ┆           ┆ not un…  │
│           ┆           ┆           ┆ Where do  ┆   ┆           ┆           ┆           ┆          │
│           ┆           ┆           ┆ …         ┆   ┆           ┆           ┆           ┆          │
└───────────┴───────────┴───────────┴───────────┴───┴───────────┴───────────┴───────────┴──────────┘

train_retrieved.shape: (106042, 18)


### Fine-tuning

In [11]:
train_dataset = Dataset.from_polars(train_retrieved)
train_dataset


Dataset({
    features: ['QuestionId', 'ConstructName', 'SubjectName', 'QuestionText', 'CorrectAnswer', 'AnswerType', 'AnswerText', 'AllText', 'AnswerAlphabet', 'QuestionId_Answer', 'MisconceptionId', 'PredictMisconceptionId', 'MisconceptionName', 'SubjectNames', 'MisconceptionName_with_SubjectNames', 'PredictMisconceptionName', 'PredictSubjectNames', 'PredictMisconceptionName_with_SubjectNames'],
    num_rows: 106042
})

In [12]:
if DEBUG:
    train_dataset = train_dataset.select(range(500))


In [13]:
# 実験結果格納用のディレクトリを作成
japan_tz = pytz.timezone("Asia/Tokyo")
cfg.run_time = datetime.now(japan_tz).strftime("%Y%m%d_%H%M%S")
create_dir(cfg.data.results_path)

model = SentenceTransformer(cfg.model.model_name, trust_remote_code=True)

loss = MultipleNegativesRankingLoss(model)

FP = False if torch.cuda.is_bf16_supported() else True
BF = True if torch.cuda.is_bf16_supported() else False
print(f"{torch.cuda.is_bf16_supported()=}")

print(f"{cfg.model.model_name}のfine-tuningを開始します。")

args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir=cfg.data.results_path,
    # Optional training parameters:
    num_train_epochs=cfg.model.epoch,
    per_device_train_batch_size=cfg.model.batch_size,
    gradient_accumulation_steps=128 // cfg.model.batch_size,
    per_device_eval_batch_size=cfg.model.batch_size,
    eval_accumulation_steps=128 // cfg.model.batch_size,
    learning_rate=cfg.model.lr,
    weight_decay=0.01,
    warmup_ratio=0.1,
    fp16=FP,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=BF,  # Set to True if you have a GPU that supports BF16
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # no duplicate samples in a batch
    # Optional tracking/debugging parameters:
    lr_scheduler_type="cosine_with_restarts",
    save_strategy="steps",
    save_steps=0.1,
    save_total_limit=2,
    logging_steps=100,
    # report_to=REPORT_TO,  # Will be used in W&B if `wandb` is installed
    # run_name=EXP_NAME,
    do_eval=False,
)

# formatted_dataset = Dataset.from_dict(
#     {
#         "anchor": train_dataset["AllText"],
#         "positive": train_dataset["MisconceptionName_with_SubjectNames"],
#         "negative": train_dataset["PredictMisconceptionName_with_SubjectNames"],
#     }
# )

trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset.select_columns(
        ["AllText", "MisconceptionName_with_SubjectNames", "PredictMisconceptionName_with_SubjectNames"]
    ),
    # train_dataset=formatted_dataset,
    loss=loss
)


trainer.train()
model.save_pretrained(cfg.data.results_path, create_model_card=False)


Directory created: ../../results/002/20241113_100343
torch.cuda.is_bf16_supported()=True
BAAI/bge-large-en-v1.5のfine-tuningを開始します。


Error while generating model card:                                   
Traceback (most recent call last):
  File "/home/marumarukun/pj/compe/kaggle_eedi/.venv/lib/python3.12/site-packages/sentence_transformers/SentenceTransformer.py", line 1233, in _create_model_card
    model_card = generate_model_card(self)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/marumarukun/pj/compe/kaggle_eedi/.venv/lib/python3.12/site-packages/sentence_transformers/model_card.py", line 962, in generate_model_card
    model_card = ModelCard.from_template(card_data=model.model_card_data, template_path=template_path, hf_emoji="🤗")
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/marumarukun/pj/compe/kaggle_eedi/.venv/lib/python3.12/site-packages/huggingface_hub/repocard.py", line 416, in from_template
    return super().from_template(card_data, template_path, template_str, **template_kwargs)
           ^^^^^^^^^^^^^^^^^